# NEURAL MACHINE TRANSLATION - LSTM

## Required Module & Config files

In [1]:
import src.LSTM as lstmNMT
from src.Tokenizer import Corpus, LangData, dataLoader
from src.utils import load_config, get_device, train_model
from src.Translator import  Translator
from torch.nn import CrossEntropyLoss
from torch.optim import NAdam
import numpy as np

In [2]:
from src.utils import torch_bleu_score

In [3]:
# Loading config file
config = load_config()
# Get device : GPU/MPS Back-End/CPU
device = get_device()
print(f"Using device: {device}")

Using device: mps


## Data Preprocessing

In [4]:
# # TRAIN_DATA
# preprocess_data(config.TRAIN_RAW, config.TRAIN_DATA, config.TRAIN_SOURCE, "english")
# preprocess_data(config.TRAIN_RAW, config.TRAIN_DATA, config.TRAIN_TARGET, "afrikaans")

# # VAL_DATA
# preprocess_data(config.VAL_RAW, config.VAL_DATA, config.VAL_SOURCE, "english")
# preprocess_data(config.VAL_RAW, config.VAL_DATA, config.VAL_TARGET, "afrikaans")

## Load the dataset

In [5]:
# Encoder-Source
english_data = Corpus(f"{config.TRAIN_DATA}/english.txt", "English")
afrikaans_data = Corpus(f"{config.TRAIN_DATA}/afrikaans.txt", "Afrikaans")

## Set Hyperparameters

In [11]:
# Encoder - source
IN_ENCODER = english_data.vocab_size
ENCODER_EMB = 128

# Decoder - target
IN_DECODER = afrikaans_data.vocab_size
OUT_DECODER = afrikaans_data.vocab_size
DECODER_EMB = 128

# Shared
HIDDEN_SIZE = 1024
NUM_LAYERS = 2

LR = 1e-3
BATCH_SIZE = 128

## Set the model

In [7]:
encoder_net = lstmNMT.Encoder(IN_ENCODER, ENCODER_EMB, HIDDEN_SIZE, NUM_LAYERS).to(device)
decoder_net = lstmNMT.Decoder(IN_DECODER, DECODER_EMB, HIDDEN_SIZE, NUM_LAYERS).to(device)
model = lstmNMT.LSTM_NMT(encoder_net, decoder_net, OUT_DECODER)

In [12]:
train_data = LangData(english_data, afrikaans_data)
train_loader = dataLoader(train_data, BATCH_SIZE)

pad_idx = afrikaans_data.stoi['<pad>']
criterion = CrossEntropyLoss(ignore_index=0)

optimizer = NAdam(model.parameters(), LR)
translator = Translator(model, english_data, afrikaans_data, device)

In [9]:
# Data used for follow-up durring training
mytext = "<sos> the classifier makes <num> correct positive predictions and <num> <com> <num> correct negative predictions <eos>"
ground = "<sos> die klassifiseerder maak <num> korrekte positiewe voorspellings en <num> <com> <num> korrekte negatiewe voorspellings <eos>"

predicted = translator.translate_sentence(mytext, method="beam")
bleu = torch_bleu_score([predicted], [ground])
print(f"Pred: {predicted}")
print(f"Refe: {ground}")
print(f"BLEU: {bleu.item()}")

Pred: <sos> vereistes deeglik enkele enkele somtyds enkele aanbeveel aanbeveel h h enkele enkele somtyds somtyds aanbeveel aanbeveel h h enkele somtyds
Refe: <sos> die klassifiseerder maak <num> korrekte positiewe voorspellings en <num> <com> <num> korrekte negatiewe voorspellings <eos>
BLEU: 0.0


## Train the data

In [19]:
EPOCHS = 20
params = {
    "model": model,
    "train_loader": train_loader,
    "optimizer": optimizer,
    "criterion": criterion,
    "device": device,
    "epochs": EPOCHS,
    "source_test": mytext,
    "target_test": ground,
	"translator":translator
}

train_model(**params)

Epoch 1/20: 100%|██████████| 4/4 [00:01<00:00,  2.03batch/s, loss=0.414]


Predicted: <sos> die volgende twee eindige lengte diskrete tyd seine is onderskeidelik <apo>n intree wat op die stelsel aangel <opn> e <cld>
Reference: <sos> die klassifiseerder maak <num> korrekte positiewe voorspellings en <num> <com> <num> korrekte negatiewe voorspellings <eos>
BLEU Score: 0.0


Epoch 2/20: 100%|██████████| 4/4 [00:01<00:00,  2.11batch/s, loss=0.613]


Predicted: <sos> die volgende twee eindige lengte diskrete tyd seine is onderskeidelik <apo>n intree wat op die stelsel aangel <opn> e <cld>
Reference: <sos> die klassifiseerder maak <num> korrekte positiewe voorspellings en <num> <com> <num> korrekte negatiewe voorspellings <eos>
BLEU Score: 0.0


Epoch 3/20: 100%|██████████| 4/4 [00:01<00:00,  2.03batch/s, loss=0.526]


...

Predicted: <sos> die volgende figuur toon die skoenlapper berekeninge van <apo>n <ltx> punt desimasie in tyd fft <com> met <ltx> as intree
Reference: <sos> die klassifiseerder maak <num> korrekte positiewe voorspellings en <num> <com> <num> korrekte negatiewe voorspellings <eos>
BLEU Score: 0.0


Epoch 19/20: 100%|██████████| 4/4 [00:01<00:00,  2.39batch/s, loss=0.349]


Predicted: <sos> die volgende figuur toon die skoenlapper berekeninge van <apo>n <ltx> punt desimasie in frekwensie fft <com> met <ltx> as intree
Reference: <sos> die klassifiseerder maak <num> korrekte positiewe voorspellings en <num> <com> <num> korrekte negatiewe voorspellings <eos>
BLEU Score: 0.0


Epoch 20/20: 100%|██████████| 4/4 [00:01<00:00,  2.29batch/s, loss=0.332]

Predicted: <sos> die volgende twee eindige lengte diskrete tyd seine is onderskeidelik <apo>n intree wat op die stelsel aangel <opn> e <cld>
Reference: <sos> die klassifiseerder maak <num> korrekte positiewe voorspellings en <num> <com> <num> korrekte negatiewe voorspellings <eos>
BLEU Score: 0.0


In [ ]:
from nltk.translate.bleu_score import sentence_bleu, modified_precision, SmoothingFunction, corpus_bleu

In [ ]:
EN_STR = [[' '.join(sent)] for sent in english_data.data_str]
AF_STR = [[' '.join(sent)] for sent in afrikaans_data.data_str]

In [ ]:
TRANSLATED = [[translate_sentece(model, sent[0], english_data, afrikaans_data, device)] for sent in EN_STR]

In [ ]:
BLEU_SCORE = [sentence_bleu([b[0].split()],a[0].split(), smoothing_function=SmoothingFunction().method1) for a,b in zip(TRANSLATED, AF_STR)]

In [ ]:
np.mean(BLEU_SCORE)

In [ ]:
from torcheval.metrics.functional import bleu_score as torchblue

In [ ]:
def torch_blue_score(candidat, reference, device=None):
    n_gram = min(len(candidat.split()), len(reference.split()), 4)
    score = torchblue(candidat, [reference], n_gram, device=device)
    return score.item()

In [ ]:
BLEU_SCORE2 = [torch_blue_score(a[0],b[0]) for a, b in zip(TRANSLATED, AF_STR)]
np.mean(BLEU_SCORE2)